# Fractional Quantum Anharmonic Oscillator (FQAHO) Simulation

This notebook implements and visualizes the FQAHO model for Choir's dynamic stake pricing.

## Key Parameters
- **α (alpha)**: Fractional parameter (1<α≤2) controlling memory effects
- **K₀**: Anharmonic coefficient representing feedback sensitivity
- **m**: Potential order representing network complexity

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.animation import FuncAnimation
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
import seaborn as sns
from scipy.special import gamma

# Set plotting style
plt.style.use('ggplot')
sns.set_context("notebook", font_scale=1.2)

# For interactive plots in Jupyter
%matplotlib inline

: 

## Model Implementation

We implement the core FQAHO functions for parameter modulation and price calculations.

In [ ]:
class FQAHOModel:
    def __init__(self, S0=1.0, delta1=0.5, delta2=0.3, tau=3.0,
                 gamma1=0.8, gamma2=0.5, beta1=0.8, beta2=0.4,
                 K0_base=1.0, c0=10.0, n0=5.0):
        """
        Initialize FQAHO model with sensitivity parameters

        Parameters:
        -----------
        S0 : float
            Base stake price
        delta1, delta2 : float
            Sensitivity parameters for alpha modulation
        tau : float
            Time constant for thread age
        gamma1, gamma2 : float
            Sensitivity parameters for K0 modulation
        beta1, beta2 : float
            Sensitivity parameters for m modulation
        K0_base : float
            Base anharmonic coefficient
        c0, n0 : float
            Reference scales for citations and co-authors
        """
        # Store parameters
        self.S0 = S0
        self.delta1 = delta1
        self.delta2 = delta2
        self.tau = tau
        self.gamma1 = gamma1
        self.gamma2 = gamma2
        self.beta1 = beta1
        self.beta2 = beta2
        self.K0_base = K0_base
        self.c0 = c0
        self.n0 = n0

    def calc_alpha(self, t, q):
        """
        Calculate fractional parameter alpha

        Parameters:
        -----------
        t : float
            Normalized thread age
        q : float
            Quality measure (0 to 1)

        Returns:
        --------
        alpha : float
            Fractional parameter (1 < alpha ≤ 2)
        """
        alpha = 2.0 - self.delta1 * (1 - np.exp(-t / self.tau)) - self.delta2 * q
        # Ensure alpha stays in valid range
        return np.clip(alpha, 1.01, 2.0)

    def calc_K0(self, r, alpha):
        """
        Calculate anharmonic coefficient K0

        Parameters:
        -----------
        r : float
            Recent refusal ratio (0 to 1)
        alpha : float
            Current fractional parameter

        Returns:
        --------
        K0 : float
            Anharmonic coefficient
        """
        K0 = self.K0_base * (1 + self.gamma1 * r) * (2.0 / alpha) ** self.gamma2
        # Ensure K0 stays in valid range
        return np.clip(K0, 0.5, 5.0)

    def calc_m(self, c, n):
        """
        Calculate potential order m

        Parameters:
        -----------
        c : float
            Citation count
        n : float
            Co-author count

        Returns:
        --------
        m : float
            Potential order (2 ≤ m ≤ 4)
        """
        m = 2.0 + self.beta1 * np.tanh(c / self.c0) + self.beta2 * np.log(1 + n / self.n0)
        # Ensure m stays in valid range
        return np.clip(m, 2.0, 4.0)

    def calc_price(self, n, alpha, K0, m, lambda_val=1.0):
        """
        Calculate stake price using FQAHO formula

        Parameters:
        -----------
        n : int
            Energy level (typically 0 for ground state)
        alpha : float
            Fractional parameter
        K0 : float
            Anharmonic coefficient
        m : float
            Potential order
        lambda_val : float
            Perturbation parameter

        Returns:
        --------
        price : float
            Calculated stake price
        """
        # Core FQAHO formula: P₀ = S₀[(2n+1)^(α/2) + (K₀λ)^{α/(m+1)}]
        quantum_term = (2 * n + 1) ** (alpha / 2)
        anharmonic_term = (K0 * lambda_val) ** (alpha / (m + 1))
        return self.S0 * (quantum_term + anharmonic_term)

    def simulate_thread(self, duration=100, seed=None):
        """
        Simulate thread evolution over time

        Parameters:
        -----------
        duration : int
            Number of time steps to simulate
        seed : int or None
            Random seed for reproducibility

        Returns:
        --------
        df : pandas.DataFrame
            DataFrame with simulation results
        """
        if seed is not None:
            np.random.seed(seed)

        # Initialize arrays to store results
        times = np.arange(duration) / duration  # Normalized time
        quality = np.zeros(duration)
        refusal_ratio = np.zeros(duration)
        citations = np.zeros(duration)
        coauthors = np.zeros(duration)
        alpha_values = np.zeros(duration)
        K0_values = np.zeros(duration)
        m_values = np.zeros(duration)
        prices = np.zeros(duration)

        # Initial values
        quality[0] = 0.5  # Start with moderate quality
        refusal_ratio[0] = 0.1  # Low initial refusal
        citations[0] = 0  # No citations initially
        coauthors[0] = 1  # Start with single author

        # Calculate initial parameters
        alpha_values[0] = self.calc_alpha(times[0], quality[0])
        K0_values[0] = self.calc_K0(refusal_ratio[0], alpha_values[0])
        m_values[0] = self.calc_m(citations[0], coauthors[0])
        prices[0] = self.calc_price(0, alpha_values[0], K0_values[0], m_values[0])

        # Simulate thread evolution
        for i in range(1, duration):
            # Simple evolution models - these would be more sophisticated in reality
            quality[i] = quality[i-1] + np.random.normal(0, 0.05)
            quality[i] = np.clip(quality[i], 0, 1)

            refusal_ratio[i] = refusal_ratio[i-1] + np.random.normal(0, 0.03)
            refusal_ratio[i] = np.clip(refusal_ratio[i], 0, 1)

            # Citations grow over time with occasional jumps
            if np.random.random() < 0.1:  # 10% chance of citation
                citations[i] = citations[i-1] + np.random.poisson(1)
            else:
                citations[i] = citations[i-1]

            # Coauthors occasionally join
            if np.random.random() < 0.05:  # 5% chance of new coauthor
                coauthors[i] = coauthors[i-1] + 1
            else:
                coauthors[i] = coauthors[i-1]

            # Calculate FQAHO parameters
            alpha_values[i] = self.calc_alpha(times[i], quality[i])
            K0_values[i] = self.calc_K0(refusal_ratio[i], alpha_values[i])
            m_values[i] = self.calc_m(citations[i], coauthors[i])

            # Calculate price
            prices[i] = self.calc_price(0, alpha_values[i], K0_values[i], m_values[i])

        # Create DataFrame with all simulation data
        return pd.DataFrame({
            'time': times,
            'quality': quality,
            'refusal_ratio': refusal_ratio,
            'citations': citations,
            'coauthors': coauthors,
            'alpha': alpha_values,
            'K0': K0_values,
            'm': m_values,
            'price': prices
        })

## Parameter Isolation Analysis

First, we'll analyze how each parameter affects the stake price independently.

In [ ]:
# Create model instance
model = FQAHOModel()

# Parameter ranges
alpha_range = np.linspace(1.01, 2.0, 100)
K0_range = np.linspace(0.5, 5.0, 100)
m_range = np.linspace(2.0, 4.0, 100)

# Fixed parameter values for isolation tests
alpha_fixed = 1.5
K0_fixed = 1.0
m_fixed = 2.5

# Calculate prices varying one parameter at a time
prices_alpha = [model.calc_price(0, alpha, K0_fixed, m_fixed) for alpha in alpha_range]
prices_K0 = [model.calc_price(0, alpha_fixed, K0, m_fixed) for K0 in K0_range]
prices_m = [model.calc_price(0, alpha_fixed, K0_fixed, m) for m in m_range]

# Plot results
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Alpha variation
axes[0].plot(alpha_range, prices_alpha, 'b-', linewidth=2)
axes[0].set_xlabel('Fractional Parameter (α)')
axes[0].set_ylabel('Stake Price')
axes[0].set_title('Price vs. α (K₀={}, m={})'.format(K0_fixed, m_fixed))
axes[0].grid(True)

# K0 variation
axes[1].plot(K0_range, prices_K0, 'r-', linewidth=2)
axes[1].set_xlabel('Anharmonic Coefficient (K₀)')
axes[1].set_ylabel('Stake Price')
axes[1].set_title('Price vs. K₀ (α={}, m={})'.format(alpha_fixed, m_fixed))
axes[1].grid(True)

# m variation
axes[2].plot(m_range, prices_m, 'g-', linewidth=2)
axes[2].set_xlabel('Potential Order (m)')
axes[2].set_ylabel('Stake Price')
axes[2].set_title('Price vs. m (α={}, K₀={})'.format(alpha_fixed, K0_fixed))
axes[2].grid(True)

plt.tight_layout()
plt.show()

## Parameter Coupling Analysis

Next, we'll explore how parameters interact with each other by creating 2D parameter slices and 3D visualizations.

In [ ]:
# Create parameter grids for 2D slices
alpha_2d = np.linspace(1.01, 2.0, 50)
K0_2d = np.linspace(0.5, 5.0, 50)
m_2d = np.linspace(2.0, 4.0, 50)

# Create meshgrids for 2D analysis
alpha_mesh, K0_mesh = np.meshgrid(alpha_2d, K0_2d)
alpha_mesh2, m_mesh = np.meshgrid(alpha_2d, m_2d)
K0_mesh2, m_mesh2 = np.meshgrid(K0_2d, m_2d)

# Calculate prices for each parameter combination
prices_alpha_K0 = np.zeros_like(alpha_mesh)
prices_alpha_m = np.zeros_like(alpha_mesh2)
prices_K0_m = np.zeros_like(K0_mesh2)

for i in range(len(K0_2d)):
    for j in range(len(alpha_2d)):
        prices_alpha_K0[i, j] = model.calc_price(0, alpha_mesh[i, j], K0_mesh[i, j], m_fixed)

for i in range(len(m_2d)):
    for j in range(len(alpha_2d)):
        prices_alpha_m[i, j] = model.calc_price(0, alpha_mesh2[i, j], K0_fixed, m_mesh[i, j])

for i in range(len(m_2d)):
    for j in range(len(K0_2d)):
        prices_K0_m[i, j] = model.calc_price(0, alpha_fixed, K0_mesh2[i, j], m_mesh2[i, j])

# Create 2D heatmap visualizations
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Alpha vs K0
im1 = axes[0].contourf(alpha_mesh, K0_mesh, prices_alpha_K0, 20, cmap='viridis')
axes[0].set_xlabel('Fractional Parameter (α)')
axes[0].set_ylabel('Anharmonic Coefficient (K₀)')
axes[0].set_title('Price Heatmap: α vs K₀ (m={})'.format(m_fixed))
fig.colorbar(im1, ax=axes[0], label='Stake Price')

# Alpha vs m
im2 = axes[1].contourf(alpha_mesh2, m_mesh, prices_alpha_m, 20, cmap='plasma')
axes[1].set_xlabel('Fractional Parameter (α)')
axes[1].set_ylabel('Potential Order (m)')
axes[1].set_title('Price Heatmap: α vs m (K₀={})'.format(K0_fixed))
fig.colorbar(im2, ax=axes[1], label='Stake Price')

# K0 vs m
im3 = axes[2].contourf(K0_mesh2, m_mesh2, prices_K0_m, 20, cmap='inferno')
axes[2].set_xlabel('Anharmonic Coefficient (K₀)')
axes[2].set_ylabel('Potential Order (m)')
axes[2].set_title('Price Heatmap: K₀ vs m (α={})'.format(alpha_fixed))
fig.colorbar(im3, ax=axes[2], label='Stake Price')

plt.tight_layout()
plt.show()

# Create 3D visualization of parameter space
fig = plt.figure(figsize=(12, 10))
ax = fig.add_subplot(111, projection='3d')

# Create a coarser grid for 3D visualization
alpha_3d = np.linspace(1.01, 2.0, 15)
K0_3d = np.linspace(0.5, 5.0, 15)
m_3d = np.linspace(2.0, 4.0, 15)

alpha_3d_mesh, K0_3d_mesh, m_3d_mesh = np.meshgrid(alpha_3d, K0_3d, m_3d)
prices_3d = np.zeros_like(alpha_3d_mesh)

# Calculate prices for the 3D grid (this is computationally intensive)
for i in range(len(K0_3d)):
    for j in range(len(alpha_3d)):
        for k in range(len(m_3d)):
            prices_3d[i, j, k] = model.calc_price(0, alpha_3d_mesh[i, j, k],
                                                 K0_3d_mesh[i, j, k], m_3d_mesh[i, j, k])

# Find stable price regions (defined as prices within a reasonable range)
# For demonstration, we'll consider prices between 0.8 and 2.0 as stable
stable_mask = (prices_3d >= 0.8) & (prices_3d <= 2.0)
stable_alpha = alpha_3d_mesh[stable_mask]
stable_K0 = K0_3d_mesh[stable_mask]
stable_m = m_3d_mesh[stable_mask]
stable_prices = prices_3d[stable_mask]

# Scatter plot of stable regions with color indicating price
scatter = ax.scatter(stable_alpha, stable_K0, stable_m, c=stable_prices,
                     cmap='viridis', alpha=0.7, s=50)

ax.set_xlabel('Fractional Parameter (α)')
ax.set_ylabel('Anharmonic Coefficient (K₀)')
ax.set_zlabel('Potential Order (m)')
ax.set_title('Stable Price Regions in Parameter Space')

# Add a color bar
cbar = fig.colorbar(scatter, ax=ax, label='Stake Price')

plt.tight_layout()
plt.show()

## Thread Evolution Simulation

Now we'll simulate different types of thread evolution and visualize parameter trajectories.

In [ ]:
# Create four different thread scenarios
# 1. New thread with steady growth
# 2. Mature thread with many citations
# 3. Controversial thread with high refusal
# 4. Breakthrough thread with rapid citation growth

# Create custom models for each scenario
model_standard = FQAHOModel()
model_mature = FQAHOModel(delta1=0.3, beta1=1.2)  # Higher citation sensitivity
model_controversial = FQAHOModel(gamma1=1.5)  # Higher refusal sensitivity
model_breakthrough = FQAHOModel(beta1=1.5, beta2=0.8)  # Higher citation and coauthor impact

# Simulate threads
df_standard = model_standard.simulate_thread(duration=200, seed=42)
df_mature = model_mature.simulate_thread(duration=200, seed=43)
df_controversial = model_controversial.simulate_thread(duration=200, seed=44)
df_breakthrough = model_breakthrough.simulate_thread(duration=200, seed=45)

# Manually adjust the mature thread to have more citations
df_mature['citations'] = np.floor(df_mature['citations'] * 3 + 10)
df_mature['coauthors'] = np.floor(df_mature['coauthors'] * 2 + 2)

# Manually adjust the controversial thread to have higher refusal ratio
df_controversial['refusal_ratio'] = df_controversial['refusal_ratio'] * 1.5 + 0.2
df_controversial['refusal_ratio'] = np.clip(df_controversial['refusal_ratio'], 0, 1)

# Manually adjust the breakthrough thread for rapid citation growth
for i in range(50, 100):
    df_breakthrough.loc[i, 'citations'] = df_breakthrough.loc[i-1, 'citations'] + np.random.poisson(1.5)

for i in range(100, 200):
    df_breakthrough.loc[i, 'citations'] = df_breakthrough.loc[i-1, 'citations'] + np.random.poisson(0.5)

# Recalculate parameters for the adjusted threads
for i in range(1, 200):
    # Mature thread
    df_mature.loc[i, 'alpha'] = model_mature.calc_alpha(df_mature.loc[i, 'time'], df_mature.loc[i, 'quality'])
    df_mature.loc[i, 'K0'] = model_mature.calc_K0(df_mature.loc[i, 'refusal_ratio'], df_mature.loc[i, 'alpha'])
    df_mature.loc[i, 'm'] = model_mature.calc_m(df_mature.loc[i, 'citations'], df_mature.loc[i, 'coauthors'])
    df_mature.loc[i, 'price'] = model_mature.calc_price(0, df_mature.loc[i, 'alpha'],
                                                       df_mature.loc[i, 'K0'], df_mature.loc[i, 'm'])

    # Controversial thread
    df_controversial.loc[i, 'alpha'] = model_controversial.calc_alpha(df_controversial.loc[i, 'time'],
                                                                     df_controversial.loc[i, 'quality'])
    df_controversial.loc[i, 'K0'] = model_controversial.calc_K0(df_controversial.loc[i, 'refusal_ratio'],
                                                               df_controversial.loc[i, 'alpha'])
    df_controversial.loc[i, 'm'] = model_controversial.calc_m(df_controversial.loc[i, 'citations'],
                                                             df_controversial.loc[i, 'coauthors'])
    df_controversial.loc[i, 'price'] = model_controversial.calc_price(0, df_controversial.loc[i, 'alpha'],
                                                                     df_controversial.loc[i, 'K0'],
                                                                     df_controversial.loc[i, 'm'])

    # Breakthrough thread
    df_breakthrough.loc[i, 'alpha'] = model_breakthrough.calc_alpha(df_breakthrough.loc[i, 'time'],
                                                                   df_breakthrough.loc[i, 'quality'])
    df_breakthrough.loc[i, 'K0'] = model_breakthrough.calc_K0(df_breakthrough.loc[i, 'refusal_ratio'],
                                                             df_breakthrough.loc[i, 'alpha'])
    df_breakthrough.loc[i, 'm'] = model_breakthrough.calc_m(df_breakthrough.loc[i, 'citations'],
                                                           df_breakthrough.loc[i, 'coauthors'])
    df_breakthrough.loc[i, 'price'] = model_breakthrough.calc_price(0, df_breakthrough.loc[i, 'alpha'],
                                                                   df_breakthrough.loc[i, 'K0'],
                                                                   df_breakthrough.loc[i, 'm'])

# Plot thread evolution metrics
fig, axes = plt.subplots(4, 4, figsize=(20, 16))

# Column labels
thread_labels = ['Standard Thread', 'Mature Thread', 'Controversial Thread', 'Breakthrough Thread']

# Row contents
dfs = [df_standard, df_mature, df_controversial, df_breakthrough]
metrics = ['citations', 'alpha', 'K0', 'm', 'price']
colors = ['b', 'g', 'r', 'purple']

for col, (df, label, color) in enumerate(zip(dfs, thread_labels, colors)):
    # Plot citations and coauthors
    ax1 = axes[0, col]
    ax1.plot(df['time'], df['citations'], 'b-', label='Citations')
    ax1.set_xlabel('Time')
    ax1.set_ylabel('Count')
    ax1.set_title(f'{label}: Citations')
    ax1.grid(True)

    # Quality and refusal ratio
    ax2 = axes[1, col]
    ax2.plot(df['time'], df['quality'], 'g-', label='Quality')
    ax2.plot(df['time'], df['refusal_ratio'], 'r--', label='Refusal Ratio')
    ax2.set_xlabel('Time')
    ax2.set_ylabel('Value')
    ax2.set_title(f'{label}: Quality & Refusal')
    ax2.legend(loc='best')
    ax2.grid(True)

    # FQAHO parameters
    ax3 = axes[2, col]
    ax3.plot(df['time'], df['alpha'], 'b-', label='α')
    ax3.plot(df['time'], df['K0'], 'r-', label='K₀')
    ax3.plot(df['time'], df['m'], 'g-', label='m')
    ax3.set_xlabel('Time')
    ax3.set_ylabel('Parameter Value')
    ax3.set_title(f'{label}: FQAHO Parameters')
    ax3.legend(loc='best')
    ax3.grid(True)

    # Stake price
    ax4 = axes[3, col]
    ax4.plot(df['time'], df['price'], color, linewidth=2)
    ax4.set_xlabel('Time')
    ax4.set_ylabel('Stake Price')
    ax4.set_title(f'{label}: Stake Price Evolution')
    ax4.grid(True)

plt.tight_layout()
plt.show()

## Parameter Trajectory Visualization

Let's visualize how the different thread types move through parameter space over time.

In [ ]:
# Create 3D visualization of parameter trajectories
fig = plt.figure(figsize=(12, 10))
ax = fig.add_subplot(111, projection='3d')

# Plot trajectories for each thread type
ax.plot(df_standard['alpha'], df_standard['K0'], df_standard['m'], 'b-',
        label='Standard Thread', linewidth=2, alpha=0.7)
ax.plot(df_mature['alpha'], df_mature['K0'], df_mature['m'], 'g-',
        label='Mature Thread', linewidth=2, alpha=0.7)
ax.plot(df_controversial['alpha'], df_controversial['K0'], df_controversial['m'], 'r-',
        label='Controversial Thread', linewidth=2, alpha=0.7)
ax.plot(df_breakthrough['alpha'], df_breakthrough['K0'], df_breakthrough['m'], 'purple',
        label='Breakthrough Thread', linewidth=2, alpha=0.7)

# Mark start and end points
for df, color in zip([df_standard, df_mature, df_controversial, df_breakthrough], ['blue', 'green', 'red', 'purple']):
    ax.scatter(df['alpha'].iloc[0], df['K0'].iloc[0], df['m'].iloc[0],
               color=color, marker='o', s=100, label='_nolegend_')
    ax.scatter(df['alpha'].iloc[-1], df['K0'].iloc[-1], df['m'].iloc[-1],
               color=color, marker='*', s=150, label='_nolegend_')

ax.set_xlabel('Fractional Parameter (α)')
ax.set_ylabel('Anharmonic Coefficient (K₀)')
ax.set_zlabel('Potential Order (m)')
ax.set_title('Thread Evolution Trajectories in FQAHO Parameter Space')
ax.legend(loc='best')

plt.tight_layout()
plt.show()

## Fractional Memory Effects

Finally, let's explore how the fractional parameter α affects memory effects in the system.

In [ ]:
def fractional_decay_kernel(t, alpha, t_max=10.0, num_points=1000):
    """
    Calculate the fractional decay kernel for different alpha values

    Parameters:
    -----------
    t : array
        Time points
    alpha : float
        Fractional parameter (1 < alpha ≤ 2)
    t_max : float
        Maximum time value
    num_points : int
        Number of time points

    Returns:
    --------
    kernel : array
        Decay kernel values
    """
    # For alpha = 2, we get exponential decay (memoryless)
    if alpha == 2.0:
        return np.exp(-t)

    # For 1 < alpha < 2, we get power law decay (long memory)
    # Using Mittag-Leffler function approximation for fractional decay
    # Note: This is a simplified approximation
    return t**(alpha-2) * np.exp(-t**(alpha/2)) / gamma(alpha/2)

# Create time points
t = np.linspace(0.01, 10, 1000)

# Calculate decay kernels for different alpha values
alpha_values = [1.2, 1.5, 1.8, 2.0]
kernels = [fractional_decay_kernel(t, alpha) for alpha in alpha_values]

# Plot decay kernels
plt.figure(figsize=(12, 6))

for alpha, kernel, color in zip(alpha_values, kernels, ['b', 'g', 'r', 'k']):
    # Normalize for better comparison
    norm_kernel = kernel / np.max(kernel)
    plt.plot(t, norm_kernel, color, linewidth=2, label=f'α = {alpha}')

plt.xlabel('Time')
plt.ylabel('Memory Decay Kernel (normalized)')
plt.title('Fractional Memory Effects in FQAHO Model')
plt.legend(loc='best')
plt.grid(True)
plt.yscale('log')
plt.xlim(0, 10)
plt.ylim(1e-3, 1)
plt.show()

# Demonstrate memory effect on pricing
def simulate_price_with_memory(model, alpha, event_time=0.2, event_size=2.0, duration=1.0, points=100):
    """
    Simulate price evolution after a significant event with memory effects

    Parameters:
    -----------
    model : FQAHOModel
        Model instance
    alpha : float
        Fractional parameter (1 < alpha ≤ 2)
    event_time : float
        When the event occurs (normalized time)
    event_size : float
        Magnitude of the event
    duration : float
        Total simulation duration
    points : int
        Number of time points

    Returns:
    --------
    times : array
        Time points
    prices : array
        Price values at each time point
    """
    times = np.linspace(0, duration, points)
    prices = np.zeros_like(times)

    # Base parameters
    K0 = 1.0
    m = 3.0

    # Find event index
    event_idx = np.argmin(np.abs(times - event_time))

    # Memory kernel for this alpha
    kernel = fractional_decay_kernel(np.linspace(0, duration, points), alpha)

    # Before event, stable price
    for i in range(event_idx):
        prices[i] = model.calc_price(0, alpha, K0, m)

    # Event and aftermath with memory effects
    for i in range(event_idx, points):
        # Calculate time since event
        dt = times[i] - times[event_idx]

        # Get memory decay factor
        if dt > 0:
            # Find closest point in kernel calculation
            decay_idx = np.argmin(np.abs(np.linspace(0, duration, len(kernel)) - dt))
            decay_factor = kernel[decay_idx] / np.max(kernel)
        else:
            decay_factor = 1.0

        # Adjust K0 based on event and memory decay
        event_K0 = K0 * (1 + event_size * decay_factor)

        # Calculate price with memory-affected K0
        prices[i] = model.calc_price(0, alpha, event_K0, m)

    return times, prices

# Simulate price evolution with different alphas
model = FQAHOModel()
memory_results = [simulate_price_with_memory(model, alpha) for alpha in alpha_values]

# Plot price evolution after event
plt.figure(figsize=(12, 6))

for alpha, (times, prices), color in zip(alpha_values, memory_results, ['b', 'g', 'r', 'k']):
    plt.plot(times, prices, color, linewidth=2, label=f'α = {alpha}')

# Mark event time
plt.axvline(x=0.2, color='gray', linestyle='--', label='Event')

plt.xlabel('Time')
plt.ylabel('Stake Price')
plt.title('Price Evolution After Event: Memory Effects for Different α Values')
plt.legend(loc='best')
plt.grid(True)
plt.show()

## Conclusion

The FQAHO model provides a sophisticated framework for dynamic stake pricing in the Choir system. Key insights from our simulation:

1. The fractional parameter α effectively controls memory effects and long-term value evolution
2. The anharmonic coefficient K₀ provides necessary feedback sensitivity to quality signals
3. The potential order m adapts to network complexity and citation patterns
4. Different thread types follow characteristic trajectories through parameter space
5. Parameter coupling creates coherent and economically sound behavior

These simulations validate the FQAHO model as a viable approach for implementing dynamic stake pricing in the Choir platform while capturing memory effects and non-local interactions.